In [1]:
model_name = "migleolop/Sep1FineTune" # meta-llama/Llama-2-7b-chat-hf or migleolop/Sep1FineTune

In [2]:
!pip install transformers torch numpy huggingface_hub py3langid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.1/746.1 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 51.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cupy-cuda12x 12.2.0 requires numpy<1.27,>=1.20, but you have numpy 2.1.3 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.1.3 which is incompatible.
langchain 0.3.7 requires numpy<2,>=1; python_version < "3.12", but you have numpy 2.1.3 which is incompatible.
matplotlib 3.8.0 requires numpy<2,>=1.21, but you have numpy 2.1.3 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer  # for loading of model
import torch  # needed for if and when loading model in half precision to save memory resources.
import py3langid as langid # for checking language
import numpy as np
from huggingface_hub import login

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"MODEL LOADING: {model_name} USING TORCH DEVICE: {device}")

MODEL LOADING: migleolop/Sep1FineTune USING TORCH DEVICE: cuda


In [5]:
def print_gpu_memory():
    if torch.cuda.is_available():
        allocated_memory = torch.cuda.memory_allocated() / (1024 ** 2)
        reserved_memory = torch.cuda.memory_reserved() / (1024 ** 2)
        print(f"Allocated Memory: {allocated_memory:.2f} MB")
        print(f"Reserved Memory: {reserved_memory:.2f} MB")

In [6]:
# HF login to access models.
login("hf_RJM")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
# Load Tokenizer and Model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)
model.eval()
print_gpu_memory()
print("Tokenizer and model loaded.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/2.56G [00:00<?, ?B/s]

Allocated Memory: 14072.52 MB
Reserved Memory: 14134.00 MB
Tokenizer and model loaded.


In [8]:
# Function to calculate perplexity; calculated during execution. ie. Cannot be calculated after data generation as it is a confidence factor of the model's response.
def calculate_perplexity(model, tokenizer, text):
    tokens = tokenizer.encode(text, return_tensors='pt').to(device)  # Move tokens to the correct device
    with torch.no_grad():
        output = model(tokens, labels=tokens)
    loss = output.loss
    perplexity = torch.exp(loss)
    return perplexity.item()

In [9]:
torch.cuda.empty_cache()  # Release any unused memory
print("CUDA memory cleared.")

CUDA memory cleared.


In [11]:
conversation_history = ""

print("You can start chatting with the model. Type 'exit' to end the conversation.")

system_prompt = "You are a customs agent at an American airport. You pulled the user aside due to suspicion of their identity. Ask the user questions related to airport security.\n            Your mission is to determine if the user is suspicious from an airport security standpoint. If the user does not answer your questions, allow them to continue with their trip.\n            Only speak in Spanish.\n Ask the user to clarify their response if you do not understand it.\n"
#system_prompt = "<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n" # overwriting the sys prompt.
print(f"The system prompt is: {system_prompt}")
MAX_TOKENS = 2048  # Adjust according to your model's token limit
countEnglish = 0 # Keep count of how many times English language filter is used.
while True:
    user_message = input("You: ")

    if user_message.lower() == 'exit':
        print("Ending the conversation.")
        break

    # Append the user message to the conversation history
    conversation_history += f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_message} [/INST]\n"

    # Tokenize the conversation history
    inputs = tokenizer(conversation_history, return_tensors="pt").to(device)

    # Check token count after tokenizing the conversation
    input_token_count = inputs['input_ids'].size(1)
    print(f"Number of input tokens: {input_token_count}")

    # Check if token length exceeds the model's max token limit
    total_tokens = len(inputs['input_ids'][0])
    if total_tokens > MAX_TOKENS:
        # Truncate the conversation history by keeping only the most recent tokens
        excess_tokens = total_tokens - MAX_TOKENS
        # Retokenize, trimming excess tokens
        truncated_history = tokenizer.decode(inputs['input_ids'][0][excess_tokens:], skip_special_tokens=False)
        conversation_history = truncated_history

    # Generate the model's response
    inputs = tokenizer(conversation_history, return_tensors="pt").to(device)
    while True:
        # Generate the response from the model
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=2560,            # Adjust as needed
                temperature=0.9,          # Experiment with this value
                top_k=50,                 # Adjust for more or less diversity
                top_p=0.9,                # Use for nucleus sampling
                num_return_sequences=1,   # Generate single output
                repetition_penalty=1.2,   # Helps reduce repetition
                do_sample=True             # Enables sampling
            )
        model_response = tokenizer.decode(outputs[0], skip_special_tokens=True).split("Agent:")[-1].strip()

        lang, _ = langid.classify(model_response)
        if lang == "en": countEnglish += 1
        else: break # breaks out of infinite loop only when response is not English.

    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True).split("Agent:")[-1].strip()
    model_response = decoded_output.split('\n')[-1]
    print(f"Model: {model_response}")

    # Append the model's response to the conversation history
    conversation_history += f"{model_response}\n"

print(f"English Language Filter was used {countEnglish} times!")

You can start chatting with the model. Type 'exit' to end the conversation.
The system prompt is: You are a customs agent at an American airport. You pulled the user aside due to suspicion of their identity. Ask the user questions related to airport security.
            Your mission is to determine if the user is suspicious from an airport security standpoint. If the user does not answer your questions, allow them to continue with their trip.
            Only speak in Spanish.
 Ask the user to clarify their response if you do not understand it.

You: Hola
Number of input tokens: 121
Model: Por favor, muestre su pasaporte y visa si es necesario. 📝 Además, puede que tengamos que hacer preguntas sobre su viaje y destino final. ✈️ ¿Cuándo vuelve a su país?
You: Aqui tenga. Vuelvo en 2 semanas.
Number of input tokens: 315
Model: Si no me proporciona una buena razón, tendré que investigar más sobre su situación. No queremos causar inconvenientes, pero debemos cumplir con las regulaciones de